# Note !

We can map these short description to its corresponding full description, as EG to be Egypt.

In [1]:
# Main libraries 
from sklearn.preprocessing import LabelEncoder
from fastapi.encoders import jsonable_encoder
from pydantic import BaseModel
from fastapi import FastAPI
from enum import Enum
import pandas as pd
import nest_asyncio
import uvicorn
import os
import io
import numpy as np
# Our files
from data_shuffling_split import *
from features_extraction import *
from data_preprocess import *
from ml_modeling import *
from keras_models import *
from configs import *

# Note !

Based on what we got from **Compare ML Models**, we end up by decide of using *Abo Bakr Word2vec model*, alongside with AdaBoost Classifier.

**We need to map the label as it numbers into corresponding class delicate do we use LabelEncoder as before**

In [2]:
number_of_features = 100
max_len_str  = 64
word2vec_path = "bakrianoo_unigram_cbow_model/full_uni_cbow_100_twitter.mdl"
word_to_vec_model = load_word2vec_model("models/word2vec/" + word2vec_path)

strat_test_set = pd.read_csv("dataset/test/strat_test_set.csv")
l_encoder = LabelEncoder()
l_encoder.fit(list(strat_test_set["dialect"]))
l_encoder.classes_

array(['AE', 'BH', 'DZ', 'EG', 'IQ', 'JO', 'KW', 'LB', 'LY', 'MA', 'OM',
       'PL', 'QA', 'SA', 'SD', 'SY', 'TN', 'YE'], dtype='<U2')

In [3]:
def classify_text(text, choosed_model, word_to_vec_model=word_to_vec_model, 
                  max_len_str=max_len_str, l_encoder=l_encoder):
    '''
    The function used to predict the tweets requested by the API.
    
    Argumen
        text              : string, the text we need to classify.
        word_to_vec_model : word2vec object, our word embedding model.
        max_len_str       : integer, The maximum number of tokens represent each text.
        l_encoder         : object, the encode of each class into some label to retrieve the class from.
    Return
        classifed_text    : dict, the returned classified text into the api request.
    '''
    # First process the text requested
    text_cleaned = clean_text(text)
    # Second tokenize that text to get the representation of each token
    tokenized_text = tokenize_using_nltk_TreebankWordTokenizer([text_cleaned])
    # Retrieve the whole text representation
    text_features = text_to_matrix_using_word2vec(word_to_vec_model, tokenized_text, max_len_str)
    
    # No prediction yet
    predicted = ''
    # Prediction with chosen ML Model
    if choosed_model == "Machine Learning Model":
        print("=====================Machine Model=====================")
        model_path    = "bakr/AdaBoostClassifier__f1_0.325_ml.sav"
        cls_model     = pickle_load_model("models/ml_models/" + model_path)
        
        # it return a list with one index to retrieve it directly
        predicted = cls_model.predict(text_features)
    elif choosed_model == "Deep Learning Model":
        print("=====================Deep Model=====================")
        model_path    = "models/dl_models/run_with_bakr_word2vec_Adam_lstm_with_batch_learning_rate=0.1__model.h5"
        text_features = text_features.reshape(text_features.shape[0], max_len_str, number_of_features)
        dl_model      = keras_load_model(model_path)
        predicted     = dl_model.predict(text_features)
        predicted     =np.argmax(predicted,axis=1)
        
    
    pred_result = l_encoder.inverse_transform(predicted)
    # Return dictionary with request text and the predicted class
    classifed_text = {
        'query': text,
        'We have predict your dialect text as: ': str(pred_result[0])
    }
    return classifed_text

In [4]:
# Assign an instance of the FastAPI class to the variable "app" to interact with the api.
app = FastAPI(title='Deploying a ML & DL Model with FastAPI')

# List available models using Enum
class Model(str, Enum):
    """The class to choose from the models we have, Enum represent fixed value that can not change"""
    ML_Model = "Machine Learning Model"
    DL_Model = "Deep Learning Model"


class Text(BaseModel):
    """
    The class used to get the text from the user, 
    """
    user_attention: str = """Put your Arabic text in empty value of key "query" below to predict it and choose which model architecture you need."""
    query         : str = ''
        

@app.get("/")
def home():
    # Once you go to this link you will see the get and post method below to trying out
    return "Congratulations! Your API is working as expected. Now head over to http://localhost:8000/docs."


# This endpoint handles all the logic necessary for the object detection to work.
# It requires the desired model and the dictionary of tweet and default class as we give default values to us
# In the api you can try other tweet from some_tweets below
@app.post("/predict") 
def prediction(model: Model, text_dict: Text):
    
    # Encode the retrived request data 
    text_dict      = jsonable_encoder(text_dict)
    # Run our model
    try:
        classifed_text = classify_text(text_dict['query'], model)
    except:
        classifed_text = {
        'Error': "Please Provide us with Arabic text in empty value of key 'query' in the Request body, Thanks ^^."
        }
    return classifed_text

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected
host = "0.0.0.0"

# uvicorn is fast Asynchronous Server Gateway Interface (ASGI) uvicorn handles the serving
# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [24245]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:56582 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56582 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:56630 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:56630 - "GET /openapi.json HTTP/1.1" 200 OK
(1, 64, 100)
(1, 6400)
[-0.2822  -0.1575   3.271   -5.48    -0.6846   1.536    0.1155   0.1368
 -2.66    -2.887    0.4595   0.6196  -0.5166   0.4082   0.3364  -1.882
 -2.713   -2.545    0.4863   0.1699   0.01108  1.388   -2.156    2.8
 -2.316    1.119   -0.357   -0.5176  -0.3162   0.2896   1.391   -0.1215
 -0.7866   1.626    2.725   -0.8965   0.9175  -1.507   -1.036   -0.629
 -0.4058   0.0749   1.287    2.66    -1.376   -1.145    2.055   -0.1971
 -0.327   -3.309  ]
=====================Machine Model=====================
INFO:     127.0.0.1:56632 - "POST /predict?model=Machine%20Learning%20Model HTTP/1.1" 200 OK


/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(1, 64, 100)
(1, 6400)
[-2.328   -0.9917  -2.174   -1.218   -0.8213   0.595    0.244   -1.181
 -2.256   -0.1447  -2.059    0.9424  -2.586    0.628   -2.98     1.889
 -4.46     1.122   -1.91     1.91    -0.02718  0.0338  -3.031    0.6924
  1.287   -0.5693  -0.07416  3.229   -4.062    0.5566   0.799   -1.186
  0.02551  3.201   -3.523    1.761   -3.514   -3.576   -3.35    -0.6567
 -2.783   -0.3848   1.451    1.062   -2.227    1.039    3.19    -1.053
  1.575   -1.143  ]
=====================Machine Model=====================
INFO:     127.0.0.1:56656 - "POST /predict?model=Machine%20Learning%20Model HTTP/1.1" 200 OK


/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


...

/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(1, 64, 100)
(1, 6400)
[ 0.477   -0.01135  0.0696  -4.164   -3.238    1.798    4.68    -1.371
  1.445    1.869    2.707   -2.088    2.012    0.5205   2.799   -0.548
  2.428   -1.763    2.543    4.87     0.2563   1.522    7.246   -3.271
  4.527   -0.3157   3.348   -2.203    2.527   -0.86    -2.062    5.32
  3.979   -0.4941  -0.0847   0.6763  -0.604   -0.9663   5.13     0.9136
  0.138   -0.6196   1.367   -4.137    0.2275   4.19     1.11    -2.193
 -0.1687  -0.1796 ]
=====================Machine Model=====================
INFO:     127.0.0.1:56818 - "POST /predict?model=Machine%20Learning%20Model HTTP/1.1" 200 OK


/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(1, 64, 100)
(1, 6400)
[ 0.477   -0.01135  0.0696  -4.164   -3.238    1.798    4.68    -1.371
  1.445    1.869    2.707   -2.088    2.012    0.5205   2.799   -0.548
  2.428   -1.763    2.543    4.87     0.2563   1.522    7.246   -3.271
  4.527   -0.3157   3.348   -2.203    2.527   -0.86    -2.062    5.32
  3.979   -0.4941  -0.0847   0.6763  -0.604   -0.9663   5.13     0.9136
  0.138   -0.6196   1.367   -4.137    0.2275   4.19     1.11    -2.193
 -0.1687  -0.1796 ]
=====================Deep Model=====================
INFO:     127.0.0.1:56820 - "POST /predict?model=Deep%20Learning%20Model HTTP/1.1" 200 OK
(1, 64, 100)
(1, 6400)
[ 2.506   -3.645   -0.4998   3.406   -0.4663   0.266    2.14    -1.289
 -2.28    -0.4846  -0.511   -1.741    0.58     2.408    0.5923   1.864
  1.01    -0.3096   1.424    1.643   -0.02928 -1.201    1.955   -2.826
 -1.78     0.1923   0.8696   3.75    -0.4248  -2.676   -1.929   -2.703
  4.13    -2.5     -1.373    0.0654  -0.1268  -0.538    0.6577   3.447
 -0.1959  

/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator LinearSVC from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/abdelrahman/.local/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 0.23.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
